In [157]:
from itertools import product
import heapq

In [158]:
class Planning_Domain(object):

    def __init__(self, actions=(), agents={}):
        self.actions = tuple(actions)
        self.agents = agents

    def generate_groundings(self, objects):
        actions = list()
        agents = self.agents.keys()
        for action in self.actions:
            parameters = [objects[t] for t in action.types]
            combinations = set()
            for p in product(product(*parameters), agents):
                parameters_set = frozenset(p)
                if action.unique and len(parameters_set) != len(p):
                    continue
                if action.same and parameters_set in combinations:
                    continue
                combinations.add(parameters_set)
                actions.append(action.grounding(*p))
        return actions

In [159]:
class Planning_Action(object):

    def __init__(self, name, parameters=(), preconditions=(), effects=(), unique=False, same=False):
        self.name = name
        if len(parameters) > 0:
            self.types, self.arg_names = zip(*parameters)
        else:
            self.types = tuple()
            self.arg_names = tuple()
        self.preconditions = preconditions
        self.effects = effects
        self.unique = unique
        self.same = same

    def grounding(self, *args):
        return GroundedPlanningAction(self, *args)

    def __str__(self):
        arglist = ', '.join(['%s' % pair for pair in zip(self.arg_names)])
        return '%s(%s)' % (self.name, arglist)


In [160]:
def flatten_nested_list(A):
    rt = []
    for i in A:
        if isinstance(i, list):
            rt.extend(flatten_nested_list(i))
        else:
            rt.append(i)
    return rt

In [161]:
def create_planning_domain(data):
    agents = dict()
    for agent, weight in data["agent"].items():
        agents[agent] = weight

    actions = list()

    for action, params in data["action"].items():
        param = list()
        for _type, variable_list in params["parameters"].items():
            for variable in variable_list:
                param.append((_type, variable))

        precon = list()
        for name, predicates in params["precondition"].items():
            precon.append(tuple(flatten_nested_list([name, predicates])))

        eff = list()
        for name, predicates in params["effect"].items():
            eff.append(tuple(flatten_nested_list([name, predicates])))

        actions.append(Action(action, param, precon, eff))
    return Planning_Domain(actions, agents)

In [176]:
class Planning_Task(object):

    def __init__(self, domain, objects, init=(), goal=()):
        self.grounded_actions = domain.generate_groundings(objects)

        predicates = list()
        functions = dict()
        for predicate in init:
            predicates.append(predicate)
        self.init = PlanningState(predicates, functions)

        self.goal = list()
        for g in goal:
            self.goal.append(g)
        self.weights = objects
        self.agents = domain.agents


In [177]:
def apply_grounding(arg_names, args):
    names = dict()
    for name, arg in zip(arg_names, args[0]):
        names[name] = arg

    def substitute_predicate(predicate):
        return predicate[0:1] + tuple(names.get(arg, arg) for arg in predicate[1:]) + (args[1],)

    return substitute_predicate


In [178]:
class GroundedPlanningAction(object):

    def __init__(self, action, *args):
        self.name = action.name
        ground = apply_grounding(action.arg_names, args)

        self.written = ground((self.name,) + action.arg_names)

        self.preconditions = list()
        for pre in action.preconditions:
            self.preconditions.append(ground(pre))
        self.effects = list()
        for effect in action.effects:
            self.effects.append(ground(effect))

    def __str__(self):
        arglist = ', '.join(map(str, self.written[1:]))
        return '%s(%s)' % (self.written[0], arglist)


In [179]:
def create_planning_problem(domain, data):
    obj = dict()
    for _type, variables in data["objects"].items():
        obj[_type] = variables

    init = list()
    for name, objects in data["init"].items():
        for object_ in objects:
            init.append(tuple(flatten_nested_list([name, object_])))

    goal = list()
    for name, objects in data["goal"].items():
        for object_ in objects:
            goal.append(tuple(flatten_nested_list([name, object_])))

    return Planning_Task(domain, obj, init, goal)


In [190]:
class PlanningState(object):

    def __init__(self, predicates, functions, predecessor=None):
        self.predicates = frozenset(predicates)
        self.functions = tuple(functions.items())
        self.f_dict = functions
        self.predecessor = predecessor

    def apply(self, action):
        no_agent_eff = [p[:-1] for p in action.effects]
        no_agent_precon = [p[:-1] for p in action.preconditions]
        new_preds = set(self.predicates)
        new_preds |= set(no_agent_eff)
        new_preds -= set(no_agent_precon)
        new_functions = dict()
        new_functions.update(self.functions)
        return PlanningState(new_preds, new_functions, (self, action))

    def extract_plan(self):
        plan = list()
        n = self
        while n.predecessor is not None:
            plan.append(n.predecessor[1])
            n = n.predecessor[0]
        plan.reverse()
        return plan

    def __hash__(self):
        return hash((self.predicates, self.functions))

    def __eq__(self, other):
        return ((self.predicates, self.functions) ==
                (other.predicates, other.functions))

    def __lt__(self, other):
        return hash(self) < hash(other)

In [191]:
def null_heuristic(state, new_state=None, goal=None):
    return 0


In [192]:
def is_true(state, predicates, problem): # NO WEIGHT
    no_agent_pred = [p[:-1] for p in predicates]
    flag = all(p in state.predicates for p in no_agent_pred) and sum(['holding' in i[0] for i in state.predicates]) < 3
    a_w = problem.agents.get(predicates[0][-1])
    b_w = [list(problem.weights.values())[0].get(p[1]) for p in predicates]
    flag2 = all(a_w >= b for b in b_w if b is not None)
    return flag + flag2


In [193]:
def equal(state, predicates):
    return all(p in state.predicates for p in predicates)

In [194]:
def h_g_heuristic(state, new_state, goal):
    h_g = 0
    for s in new_state.predicates:
        if s in state.predicates:
            h_g += 1
        if s in goal:
            h_g += 1
    return h_g


In [205]:
def astar(problem, heuristic=null_heuristic, state0=None, goal=None):
    if heuristic is None:
        heuristic = null_heuristic
    if state0 is None:
        state0 = problem.init
    if goal is None:
        goal = tuple(problem.goal)

    equal(state0, state0.predicates)
    print()

    closed = set()
    fringe = [(heuristic(state0, PlanningState(list(), dict()), goal), state0)]
    heapq.heapify(fringe)
    while True:
        if len(fringe) == 0:
            return None

        h, node = heapq.heappop(fringe)
        tmp = [p[:-1] for p in node.predicates]

        if equal(node, goal):
            plan = node.extract_plan()
            return plan

        if node not in closed:
            closed.add(node)
            successors = set(node.apply(action)
                             for action in problem.grounded_actions
                             if is_true(node, action.preconditions, problem))
            for successor in successors:
                if successor not in closed:
                    heapq.heappush(fringe, (heuristic(node, successor, goal), successor))


In [206]:
import json
with open('domain_weighted.json', 'r') as domain_file:
    domain_data = json.load(domain_file)

with open('Task1_weighted.json', 'r') as task_file:
    task_data = json.load(task_file)

In [207]:
# Create planning domain and problem
planning_domain = create_planning_domain(domain_data)
planning_problem = create_planning_problem(planning_domain, task_data)

In [208]:

print("Actions:")
[print(i.__str__()) for i in planning_domain.actions]


Actions:
pick-up(x)
put-down(x)
stack(x, y)
unstack(x, y)


[None, None, None, None]

In [209]:

print("\nAction combinations:")
[print(i.__str__()) for i in planning_problem.grounded_actions]




Action combinations:
pick-up(D, agent1)
pick-up(D, agent2)
pick-up(D, agent3)
pick-up(B, agent1)
pick-up(B, agent2)
pick-up(B, agent3)
pick-up(A, agent1)
pick-up(A, agent2)
pick-up(A, agent3)
pick-up(C, agent1)
pick-up(C, agent2)
pick-up(C, agent3)
put-down(D, agent1)
put-down(D, agent2)
put-down(D, agent3)
put-down(B, agent1)
put-down(B, agent2)
put-down(B, agent3)
put-down(A, agent1)
put-down(A, agent2)
put-down(A, agent3)
put-down(C, agent1)
put-down(C, agent2)
put-down(C, agent3)
stack(D, D, agent1)
stack(D, D, agent2)
stack(D, D, agent3)
stack(D, B, agent1)
stack(D, B, agent2)
stack(D, B, agent3)
stack(D, A, agent1)
stack(D, A, agent2)
stack(D, A, agent3)
stack(D, C, agent1)
stack(D, C, agent2)
stack(D, C, agent3)
stack(B, D, agent1)
stack(B, D, agent2)
stack(B, D, agent3)
stack(B, B, agent1)
stack(B, B, agent2)
stack(B, B, agent3)
stack(B, A, agent1)
stack(B, A, agent2)
stack(B, A, agent3)
stack(B, C, agent1)
stack(B, C, agent2)
stack(B, C, agent3)
stack(A, D, agent1)
stack(A, D

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [210]:

print("\nPlan Astar:")
[print(i.__str__()) for i in astar(problem)]


Plan Astar:

stack(B, C, agent2)
stack(C, A, agent2)
stack(D, B, agent1)
stack(C, D, agent2)
stack(D, C, agent2)
stack(A, A, agent1)
unstack(B, D, agent1)
pick-up(C, agent2)
put-down(A, agent1)
stack(B, D, agent1)
stack(A, D, agent1)
unstack(A, B, agent1)
stack(D, B, agent1)
stack(D, A, agent1)
unstack(B, B, agent1)
pick-up(D, agent1)
stack(B, A, agent1)
unstack(D, A, agent1)
stack(C, B, agent2)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]